In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time
import xml.etree.ElementTree as ET
import math
import re
import glob
import random
import string
from IPython.display import clear_output
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk
%matplotlib inline

In [5]:
df = pd.read_csv("imdb_master.csv",encoding="latin-1")
df.head(10)

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt
5,5,test,"A funny thing happened to me while watching ""M...",neg,10004_2.txt
6,6,test,This German horror film has to be one of the w...,neg,10005_2.txt
7,7,test,"Being a long-time fan of Japanese film, I expe...",neg,10006_2.txt
8,8,test,"""Tokyo Eyes"" tells of a 17 year old Japanese g...",neg,10007_4.txt
9,9,test,Wealthy horse ranchers in Buenos Aires have a ...,neg,10008_4.txt


In [6]:
df = df[df['type']=='train'].reset_index(drop=True)

In [7]:
len(df)

75000

In [8]:
train_data = df[0:1000]
train_data.head()

,Unnamed: 0,type,review,label,file
0,25000,train,Story of a man who has unnatural feelings for ...,neg,0_3.txt
1,25001,train,Airport '77 starts as a brand new luxury 747 p...,neg,10000_4.txt
2,25002,train,This film lacked something I couldn't put my f...,neg,10001_4.txt
3,25003,train,"Sorry everyone,,, I know this is supposed to b...",neg,10002_1.txt
4,25004,train,When I was little my parents took me along to ...,neg,10003_1.txt


In [9]:
train_data.columns = ['id', 'dataset','text','pol','file']
train_data.head()

,id,dataset,text,pol,file
0,25000,train,Story of a man who has unnatural feelings for ...,neg,0_3.txt
1,25001,train,Airport '77 starts as a brand new luxury 747 p...,neg,10000_4.txt
2,25002,train,This film lacked something I couldn't put my f...,neg,10001_4.txt
3,25003,train,"Sorry everyone,,, I know this is supposed to b...",neg,10002_1.txt
4,25004,train,When I was little my parents took me along to ...,neg,10003_1.txt


In [10]:
train_data['text'][0].split('.')

['Story of a man who has unnatural feelings for a pig',
 ' Starts out with a opening scene that is a terrific example of absurd comedy',
 " A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers",
 ' Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting',
 ' Even those from the era should be turned off',
 ' The cryptic dialogue would make Shakespeare seem easy to a third grader',
 " On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond",
 ' Future stars Sally Kirkland and Frederic Forrest can be seen briefly',
 '']

In [11]:
train_data['text'][0].split('.')[0]


'Story of a man who has unnatural feelings for a pig'

In [12]:
len(train_data['text'][0].split('.'))

9

In [13]:
train_data_sent = pd.DataFrame()
start_time = time.time()
for index in train_data.index:
    data_row = train_data.iloc[index,:]
    for sent_id in range(0,len(data_row['text'].split('.'))-1):
        sentence = data_row['text'].split('.')[sent_id]
        sentence_row = pd.DataFrame({
            'id':data_row['id'],
            'pol':data_row['pol'],
            'sent_id':sent_id,
            'sentence':sentence}, index = [index])
        train_data_sent = train_data_sent.append(sentence_row)
        clear_output(wait=True)
        print('Proportion of comments completed:',np.round(index/len(train_data),4)*100,'%')
        end_time = time.time()
        print('Total time = ',np.round(end_time-start_time,2)/60,'minutes')
        train_data_sent = train_data_sent.reset_index(drop=True)
        

Proportion of comments completed: 99.9 %
Total time =  1.4475 minutes


In [14]:
train_data_sent['sentence_clean']= train_data_sent['sentence'].str.replace('[{}]'.format(string.punctuation), '')
train_data_sent['sentence_clean']=train_data_sent['sentence_clean'].str.lower()
train_data_sent['sentence_clean']='<s' +train_data_sent['sentence_clean']
train_data_sent['sentence_clean']=train_data_sent['sentence_clean'] + '/s>'
train_data_sent.head()

,id,pol,sent_id,sentence,sentence_clean
0,25000,neg,0,Story of a man who has unnatural feelings for ...,<sstory of a man who has unnatural feelings fo...
1,25000,neg,1,Starts out with a opening scene that is a ter...,<s starts out with a opening scene that is a t...
2,25000,neg,2,A formal orchestra audience is turned into an...,<s a formal orchestra audience is turned into ...
3,25000,neg,3,Unfortunately it stays absurd the WHOLE time ...,<s unfortunately it stays absurd the whole tim...
4,25000,neg,4,Even those from the era should be turned off,<s even those from the era should be turned of...


In [15]:
text = train_data_sent['sentence_clean']
text_list = "".join(map(str ,text))
text_list[0:100]

'<sstory of a man who has unnatural feelings for a pig/s><s starts out with a opening scene that is a'

In [20]:
word_list = pd.DataFrame({'words':text.str.split(' ', expand = True).stack().unique()})

In [21]:
word_count_table = pd.DataFrame()
for n,word in enumerate(word_list['words']):
    word_count = len(re.findall(' ' + word + ' ',text_list))
    word_count_table = word_count_table.append(pd.DataFrame({'count':word_count}, index=[n]))
    clear_output(wait=True)
    print('Proportion of words completed:',np.round(n/len(word_list),4)*100,'%')
    word_list['count'] = word_count_table['count']
    word_list['count'] = np.where(word_list['words'] == '<s' , 0,np.where(word_list['words'] == '/s>',0,word_list['count']))
    

Proportion of words completed: 100.0 %


In [22]:
word_list['prob'] = word_list['count']/sum(word_list['count'])
word_list.head()

,words,count,prob
0,<sstory,0,0.000000
1,of,5142,0.025349
2,a,5865,0.028913
3,man,119,0.000587
4,who,713,0.003515


### Unigram Model

In [23]:
unigram_table = pd.DataFrame()

start_time = time.time()
# Loop through each sentence
# REMOVE ROW LIMIT FOR FULL RUN
for index in train_data_sent[0:200].index:
    data_row = train_data_sent.iloc[index,:]

    sent_probs = pd.DataFrame()
    # Go through each word in the sentence, lookup the probability of the word and 
    # then find the mulitplicitive product of all probabilities in the sentence.
    for n,word in enumerate(data_row['sentence_clean']):
        sent_probs = sent_probs.append(pd.DataFrame({'prob':word_list[ word_list['words']==word]['prob']}, index = [n]))
    unigram = sent_probs['prob'].prod(axis=0)
    
    # Create a list of unigram calculation for each sentence
    unigram_table = unigram_table.append(pd.DataFrame({'unigram':unigram},index = [index]))
    
    clear_output(wait=True)
    print('Proportion of sentences completed:', np.round(index/len(train_data_sent),4)*100,'%')
        
end_time = time.time()
print('Total run time = ', np.round(end_time-start_time,2)/60, ' minutes')

train_data_sent['unigram'] = unigram_table['unigram']


Proportion of sentences completed: 1.49 %
Total run time =  1.3784999999999998  minutes


In [26]:
word_1 = 'to'
word_2 = 'a'

prob_word_1 = word_list[word_list['words'] == word_1]['prob'].iloc[0]
prob_word_2 = word_list[word_list['words'] == word_2]['prob'].iloc[0]

unigram_prob = prob_word_1*prob_word_2

print('The unigram probability of the word "a" occuring given the word "to" was the previous word is: ', np.round(unigram_prob,10))

The unigram probability of the word "a" occuring given the word "to" was the previous word is:  0.0007326239


## Bigram Model

In [27]:


word_1 = ' ' + str('to') + ' '
word_2 = str('a') + ' '

bigram_prob = len(re.findall(word_1 + word_2, text_list)) / len(re.findall(word_1, text_list)) 

print('The probability of the word "a" occuring given the word "to" was the previous word is: ', np.round(bigram_prob,5))



The probability of the word "a" occuring given the word "to" was the previous word is:  0.02237


In [28]:
word_1 = ' ' + str('has') + ' '
word_2 = str('a') + ' '

bigram_prob = len(re.findall(word_1 + word_2, text_list)) / len(re.findall(word_1, text_list)) 

print('The probability of the word "a" occuring given the word "has" was the previous word is: ', np.round(bigram_prob,5))


The probability of the word "a" occuring given the word "has" was the previous word is:  0.13462


In [29]:
W_W_Matrix = pd.DataFrame({'words': word_list['words']})

start_time = time.time()


# Add limits to number of columns/rows so this doesn't run for ages
column_lim = 1000
#column_lim = len(W_W_Matrix)
row_lim = 10
#row_lim = len(W_W_Matrix)

for r, column in enumerate(W_W_Matrix['words'][0:column_lim]):
    
    prob_table = pd.DataFrame()
    for i, row in enumerate(W_W_Matrix['words'][0:row_lim]):

        word_1 = ' ' + str(row) + ' '
        word_2 = str(column) + ' '

        if len(re.findall(word_1, text_list)) == 0:
            prob = pd.DataFrame({'prob':[0]}, index=[i])
        else:
            prob = pd.DataFrame({'prob':[len(re.findall(word_1 + word_2, text_list)) / len(re.findall(word_1, text_list)) ]}, index=[i])
        
        prob_table = prob_table.append(prob)
    W_W_Matrix[str(column)] = prob_table['prob']
    
    # Outputs progress of main loop, see:
    clear_output(wait=True)
    print('Proportion of column words completed:', np.round(r/len(W_W_Matrix[0:column_lim]),2)*100,'%')
    
end_time = time.time()
print('Total run time = ', np.round(end_time-start_time,2)/60, ' minutes')

Proportion of column words completed: 100.0 %
Total run time =  1.7746666666666668  minutes


In [30]:
W_W_Matrix[W_W_Matrix['a'] >= 0]

,words,<sstory,of,a,man,who,has,unnatural,feelings,for,...,hopefully,effort,produces,results/s>,bad/s>,way,weak,outdated,country/s>,walker
0,<sstory,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
1,of,0.0,0.000194,0.049592,0.00000,0.000778,0.000000,0.000000,0.0,0.000000,...,0.0,0.000389,0.0,0.0,0.0,0.000583,0.000194,0.0,0.0,0.000000
2,a,0.0,0.000000,0.000171,0.00324,0.000000,0.000000,0.000000,0.0,0.000171,...,0.0,0.000000,0.0,0.0,0.0,0.001876,0.001364,0.0,0.0,0.000341
3,man,0.0,0.000000,0.000000,0.00000,0.134454,0.008403,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
4,who,0.0,0.000000,0.001403,0.00000,0.000000,0.039271,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
5,has,0.0,0.000000,0.134615,0.00000,0.000000,0.000000,0.001923,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
6,unnatural,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.5,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
7,feelings,0.0,0.222222,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.444444,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
8,for,0.0,0.000000,0.097826,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
9,pig/s>,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000


## Trigram

In [31]:
word_1 = str('to')
word_2 = str('a') 
word_3 = str('movie')

trigram_prob = (len(re.findall(' ' + word_1 + ' ' + word_2 + ' ' + word_3 + ' ', text_list)) / 
                    len(re.findall(' ' + word_1 + ' ' + word_2, text_list)))


print('The probability of the word "',word_3,'" occuring given the word "',word_1,'" and "',word_2,'" were the previous two words is: ', (trigram_prob))



The probability of the word " movie " occuring given the word " to " and " a " were the previous two words is:  0.011799410029498525


In [32]:
word_1 = str('to')
word_2 = str('a') 
word_3 = str('film')

trigram_prob = (len(re.findall(' ' + word_1 + ' ' + word_2 + ' ' + word_3 + ' ', text_list)) / 
                    len(re.findall(' ' + word_1 + ' ' + word_2, text_list)))


print('The probability of the word "',word_3,'" occuring given the word "',word_1,'" and "',word_2,'" were the previous two words is: ', (trigram_prob))

The probability of the word " film " occuring given the word " to " and " a " were the previous two words is:  0.0058997050147492625


## Finding Entropy

In [33]:
def entropy(sentence, data_prob):
    entropy_table = pd.DataFrame()
    for n,word in enumerate(sentence.split(' ')):
        # log2(0) provide nan so return 0 instead
        if ((data_prob[data_prob['words']==word]['prob'].iloc[0]) == 0):
            entropy = 0
        else:
            prob = data_prob[data_prob['words']==word]['prob'].iloc[0]
            entropy = prob*np.log2(prob)
        entropy_table = entropy_table.append(pd.DataFrame({'word':word,
                                                            'entropy':entropy}, index = [n]))
    phrase_entropy = -1*sum(entropy_table['entropy'])
    return(phrase_entropy)

In [35]:
sent_1 = text.iloc[0]
sent_2 = text.iloc[1]

print('Sentence 1', sent_1)
print('--.--.--.--.--.--.--.--')
print('Sentence 2', sent_2)

Sentence 1 <sstory of a man who has unnatural feelings for a pig/s>
--.--.--.--.--.--.--.--
Sentence 2 <s starts out with a opening scene that is a terrific example of absurd comedy/s>


In [37]:
data_prob = word_list[['words','count','prob']]
data_prob.head()

,words,count,prob
0,<sstory,0,0.000000
1,of,5142,0.025349
2,a,5865,0.028913
3,man,119,0.000587
4,who,713,0.003515


In [38]:
sent_1_entropy = entropy(sent_1,data_prob)
sent_2_entropy = entropy(sent_2,data_prob)

print('Sentence 1: ', sent_1)
print('Sentence 1 entropy: ', np.round(sent_1_entropy,5))
print('Per-word Sentence 1 entropy: ', np.round(sent_1_entropy/len(sent_1.split(' ')),5))

print('Sentence 2: ', sent_2)
print('Sentence 2 entropy: ', np.round(sent_2_entropy,5))
print('Per-word Sentence 2 entropy: ', np.round(sent_2_entropy/len(sent_2.split(' ')),5))


Sentence 1:  <sstory of a man who has unnatural feelings for a pig/s>
Sentence 1 entropy:  0.54195
Per-word Sentence 1 entropy:  0.04927
Sentence 2:  <s starts out with a opening scene that is a terrific example of absurd comedy/s>
Sentence 2 entropy:  0.71513
Per-word Sentence 2 entropy:  0.04768


### Finding Perplexity

In [39]:
sent_1_perplex = 2**sent_1_entropy
sent_2_perplex = 2**sent_2_entropy

print('Sentence 1: ', sent_1)
print('Sentence 1 entropy: ', np.round(sent_1_entropy,5))
print('Per-word Sentence 1 entropy: ', np.round(sent_1_entropy/len(sent_1.split(' ')),5))
print('Sentence 1 Perplexity: ', sent_1_perplex)

print('--.--.--.--.--.--.--.--')
print('Sentence 2: ', sent_2)
print('Sentence 2 entropy: ', np.round(sent_2_entropy,5))
print('Per-word Sentence 2 entropy: ', np.round(sent_2_entropy/len(sent_2.split(' ')),5))
print('Sentence 2 Perplexity: ', sent_2_perplex)

Sentence 1:  <sstory of a man who has unnatural feelings for a pig/s>
Sentence 1 entropy:  0.54195
Per-word Sentence 1 entropy:  0.04927
Sentence 1 Perplexity:  1.4559422716148382
--.--.--.--.--.--.--.--
Sentence 2:  <s starts out with a opening scene that is a terrific example of absurd comedy/s>
Sentence 2 entropy:  0.71513
Per-word Sentence 2 entropy:  0.04768
Sentence 2 Perplexity:  1.6416350090277736


In [40]:
sent_1_prob = (1/sent_1_perplex)**len(sent_1.split(' '))
sent_2_prob = (1/sent_2_perplex)**len(sent_2.split(' '))

print('Sentence 1 Probability: ', '%0.10f' % sent_1_prob)
print('Sentence 2 Probability: ', '%0.10f' % sent_2_prob  )

Sentence 1 Probability:  0.0160477552
Sentence 2 Probability:  0.0005899985
